<a href="https://colab.research.google.com/github/VitorSRamos/CNN-MorfGal/blob/main/download_imagens/formato_FITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook para gerar um conjunto de imagens no formato FITS nas bandas GRI usando o catálogo disponibilizado em https://github.com/cdebom/splus_morph/blob/main/catalogs/ReportSplusMorfoNewClassV2.txt

In [ ]:
pip install splusdata

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import splusdata
from astropy.io import fits
import os

conn = splusdata.connect('username', 'password') ## from splus.cloud

In [ ]:
paper_data = pd.read_csv('/content/drive/MyDrive/.../ReportSplusMorfoNewClassV2.txt') # Local do catálogo

display(paper_data)

,ID,RA,Dec,mag_r,True_Class,Prediction_Elliptical,Prediction_Spiral,predBin_Elliptical,predBin_Spiral,Train
0,SPLUS.STRIPE82-0001.03217.griz,359.9707,-1.2178,16.50,Spiral,0.001357,0.740537,0,1,0
1,SPLUS.STRIPE82-0001.03521.griz,0.0559,-1.2136,16.34,Spiral,0.000345,0.874112,0,1,0
2,SPLUS.STRIPE82-0001.03897.griz,0.2156,-1.1983,16.77,Elliptical,0.999987,0.000018,1,0,0
3,SPLUS.STRIPE82-0001.04261.griz,359.7408,-1.1574,16.05,Spiral,0.000538,0.822629,0,1,0
4,SPLUS.STRIPE82-0001.05263.griz,359.7357,-1.1286,16.93,Spiral,0.002476,0.809346,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4227,SPLUS.STRIPE82-0170.26359.griz,359.1411,0.9549,16.96,Spiral,0.001186,0.965720,0,1,1
4228,SPLUS.STRIPE82-0170.27075.griz,358.1667,0.9403,16.87,Elliptical,0.999426,0.000196,1,0,1
4229,SPLUS.STRIPE82-0170.27428.griz,358.4245,0.9168,16.56,Spiral,0.000116,0.997570,0,1,1
4230,SPLUS.STRIPE82-0170.28231.griz,357.9039,0.8854,15.10,Elliptical,0.999625,0.000566,1,0,1


In [ ]:
folder_path = '/content/drive/MyDrive/.../img_data/' # Local de armazenamento das imagens

filenames = os.listdir(folder_path)
#len(filenames)
print(filenames[5])
filenames = [item[:-5] for item in filenames] # limpando extensão dos nomes 
print(filenames[5])

SPLUS.STRIPE82-0118.09334.griz.fits
SPLUS.STRIPE82-0118.09334.griz


In [ ]:
counter = len(filenames)
bands = ['I', 'R', 'G']
RADIUS = 256

for key, value in paper_data.iterrows():
    if value.ID in filenames:
        print(value.ID, 'já está na pasta')
    else:
        print('Gerando {}'.format(value.ID))
        # Pegandos hdu_lists de splus.cloud
        hdu_r = conn.get_cut(value.RA, value.Dec, RADIUS, bands[0])
        hdu_g = conn.get_cut(value.RA, value.Dec, RADIUS, bands[1])
        hdu_b = conn.get_cut(value.RA, value.Dec, RADIUS, bands[2])
        
        # Extraindo dados de imagem
        image_data_r = hdu_r[1].data
        image_data_g = hdu_g[1].data
        image_data_b = hdu_b[1].data

        # Criando novo arquivo FITS
        final_image_hdu_r = fits.ImageHDU(image_data_r)
        final_image_hdu_g = fits.ImageHDU(image_data_g)
        final_image_hdu_b = fits.ImageHDU(image_data_b)
        
        # Criando meu header
        hdr = fits.Header()

        hdr['RA'] = value.RA
        hdr['DEC'] = value.Dec
        hdr['RADIUS'] = RADIUS
        hdr['ID'] = value.ID
        hdr['ELLIPTICAL'] = value.predBin_Elliptical
        hdr['SPIRAL'] = value.predBin_Spiral

        empty_primary = fits.PrimaryHDU(header=hdr)

        # Juntando em uma HDU_list
        hdul = fits.HDUList([empty_primary, final_image_hdu_r, final_image_hdu_g, final_image_hdu_b])

        #salvando a imagem
        hdul.writeto(folder_path + '{}.fits'.format(value.ID))

        # Barra de progresso
        print(value.RA, value.Dec, '{}%'.format(counter*100/len(paper_data)))
        counter += 1

SPLUS.STRIPE82-0001.03217.griz já está na pasta
SPLUS.STRIPE82-0001.03521.griz já está na pasta
SPLUS.STRIPE82-0001.03897.griz já está na pasta
SPLUS.STRIPE82-0001.04261.griz já está na pasta
SPLUS.STRIPE82-0001.05263.griz já está na pasta
SPLUS.STRIPE82-0001.06137.griz já está na pasta
SPLUS.STRIPE82-0001.08015.griz já está na pasta
SPLUS.STRIPE82-0001.08226.griz já está na pasta
SPLUS.STRIPE82-0001.08662.griz já está na pasta
SPLUS.STRIPE82-0001.09777.griz já está na pasta
SPLUS.STRIPE82-0001.11896.griz já está na pasta
SPLUS.STRIPE82-0001.12430.griz já está na pasta
SPLUS.STRIPE82-0001.12998.griz já está na pasta
SPLUS.STRIPE82-0001.13351.griz já está na pasta
SPLUS.STRIPE82-0001.13589.griz já está na pasta
SPLUS.STRIPE82-0001.14080.griz já está na pasta
SPLUS.STRIPE82-0001.14771.griz já está na pasta
SPLUS.STRIPE82-0001.15853.griz já está na pasta
SPLUS.STRIPE82-0001.17570.griz já está na pasta
SPLUS.STRIPE82-0001.18039.griz já está na pasta
SPLUS.STRIPE82-0001.18456.griz já está n

# Removendo label error

In [ ]:
label_error = paper_data[(paper_data['predBin_Spiral'] == 0) & (paper_data['predBin_Elliptical'] == 0)]

display(label_error)

,ID,RA,Dec,mag_r,True_Class,Prediction_Elliptical,Prediction_Spiral,predBin_Elliptical,predBin_Spiral,Train
46,SPLUS.STRIPE82-0002.17282.griz,359.5093,0.6909,16.69,Spiral,0.667094,0.718030,0,0,0
58,SPLUS.STRIPE82-0002.29115.griz,359.6730,0.8950,16.42,Spiral,0.000850,0.499860,0,0,0
99,SPLUS.STRIPE82-0004.01971.griz,0.7441,0.1420,16.61,Elliptical,0.445510,0.132116,0,0,0
209,SPLUS.STRIPE82-0007.04186.griz,4.0183,-1.1887,16.69,Elliptical,0.871591,0.598708,0,0,0
217,SPLUS.STRIPE82-0007.09196.griz,4.2866,-0.9580,14.77,Spiral,0.000304,0.444738,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3990,SPLUS.STRIPE82-0160.03878.griz,350.9947,0.2815,16.17,Elliptical,0.479396,0.438578,0,0,1
4039,SPLUS.STRIPE82-0162.17996.griz,353.2209,1.1995,14.94,Elliptical,0.781259,0.557441,0,0,1
4061,SPLUS.STRIPE82-0163.22198.griz,354.6170,-0.3234,15.60,Spiral,0.001100,0.528487,0,0,1
4083,SPLUS.STRIPE82-0164.11883.griz,354.4338,0.3579,15.07,Spiral,0.000005,0.396876,0,0,1


In [ ]:
for key, value in label_error.iterrows():
    filename = value.ID + '.fits'
    if value.ID in filenames:
        os.remove(folder_path + filename)

In [ ]:
print(len(filenames))
clean_filenames = os.listdir(folder_path)
print(len(clean_filenames))
print(len(filenames) - len(clean_filenames))

4236
4133
103
